In [2]:
import pandas as pd
import matplotlib.pyplot as plt

/tmp/ipykernel_168291/1492400551.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [15]:
actual_data = pd.read_excel('input/Данные с 2022-wk40 по 2024-wk5.xlsx', skiprows=[0])
actual_data = actual_data.rename(columns = {'Unnamed: 0': 'date', 'ОРВИ': 'ari', 
                                            'Население': 'population', 
                                            'Число образцов тестированных на грипп': 'tested',
                                            'A (субтип не определен)': 'A_non_defined',
                                            })
actual_data.columns
actual_data

,date,Всего,Всего.1,Unnamed: 3,Unnamed: 4,Всего.2,Всего.3,Всего.4,Всего.5,Всего.6,Всего.7,Всего.8,Всего.9,Всего.10,Всего.11,Всего.12,Всего.13,Всего.14,Всего.15,Всего.16
0,2022.40 (03.10.2022 - 09.10.2022),398060,54356783,NaN,2022.40 (03.10.2022 - 09.10.2022),3175,0,0,0,0,3110,70,65,34,307,8,29,28,14175,1523
1,2022.41 (10.10.2022 - 16.10.2022),370757,54333451,NaN,2022.41 (10.10.2022 - 16.10.2022),2967,0,2,1,0,2944,94,57,53,283,6,28,24,14234,1628
2,2022.42 (17.10.2022 - 23.10.2022),364774,54322099,NaN,2022.42 (17.10.2022 - 23.10.2022),3396,1,0,0,4,3281,87,86,71,294,15,53,38,13343,765
3,2022.43 (24.10.2022 - 30.10.2022),355948,54330613,NaN,2022.43 (24.10.2022 - 30.10.2022),3150,1,4,0,0,3075,69,78,91,214,16,84,43,11772,981
4,2022.44 (31.10.2022 - 06.11.2022),312884,54330613,NaN,2022.44 (31.10.2022 - 06.11.2022),3362,0,4,0,2,3094,67,81,101,184,19,86,43,9416,618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2024.1 (01.01.2024 - 07.01.2024),260513,54070728,NaN,2024.1 (01.01.2024 - 07.01.2024),6415,443,0,948,7,6194,37,120,114,192,232,69,55,7627,466
66,2024.2 (08.01.2024 - 14.01.2024),463513,54437820,NaN,2024.2 (08.01.2024 - 14.01.2024),10721,715,1,1335,12,10557,94,129,216,230,315,159,92,11958,625
67,2024.3 (15.01.2024 - 21.01.2024),440771,54743508,NaN,2024.3 (15.01.2024 - 21.01.2024),11760,1029,1,1083,12,14286,105,152,266,234,337,184,101,11632,625
68,2024.4 (22.01.2024 - 28.01.2024),450254,54743508,NaN,2024.4 (22.01.2024 - 28.01.2024),13075,1026,58,1205,21,12679,61,148,337,251,401,204,94,13341,764


In [14]:
actual_data_preprocessed = actual_data[['date', 'ari', 'population', 'tested', 
                                        'A_non_defined', 'A(H1)pdm09', 'A(H3)', 'B']]
actual_data_preprocessed['date'] = actual_data_preprocessed['date'].str.replace(u'\xa0', u' ')
actual_data_preprocessed

KeyError: "['ari', 'population', 'tested', 'A_non_defined', 'A(H1)pdm09', 'A(H3)', 'B'] not in index"

In [3]:
actual_data_preprocessed = actual_data[['date', 'ari', 'population', 'tested', 
                                        'A_non_defined', 'A(H1)pdm09', 'A(H3)', 'B']]
actual_data_preprocessed['date'] = actual_data_preprocessed['date'].str.replace(u'\xa0', u' ')
dates = list(actual_data_preprocessed['date'])
dates = [elem.replace(u'\xa0', u' ') for elem in dates]
dates = [elem.split('(')[0] for elem in dates]
years = [int(elem.split('.')[0]) for elem in dates]
weeks = [int(elem.split('.')[1]) for elem in dates]
actual_data_preprocessed.insert(loc=0, column='num_of_week', value=weeks)
actual_data_preprocessed.insert(loc=0, column='year', value=years)
actual_data_preprocessed['date']
actual_data_final = actual_data_preprocessed.drop(columns=['date'])


def divide_A_non_defined_to_H1(row):
    if row['A(H1)pdm09'] + row['A(H3)'] == 0:
        return 0
    else:
        val = row['A(H1)pdm09'] + row['A_non_defined']*\
        row['A(H1)pdm09']/(row['A(H1)pdm09'] + row['A(H3)'])
        return val
    
    
def divide_A_non_defined_to_H3(row):
    if row['A(H1)pdm09'] + row['A(H3)'] == 0:
        return 0
    else:
        val = row['A(H3)'] + row['A_non_defined']*\
        row['A(H3)']/(row['A(H1)pdm09'] + row['A(H3)'])
        return val
    
actual_data_final['A(H1)pdm09_abs'] = actual_data_final.apply(divide_A_non_defined_to_H1, axis=1)
actual_data_final['A(H3)_abs'] = actual_data_final.apply(divide_A_non_defined_to_H3, axis=1)
actual_data_final['B_abs'] = actual_data_final['B']
actual_data_final['A(H1)pdm09_rel'] = actual_data_final['A(H1)pdm09_abs']/actual_data_final['tested']
actual_data_final['A(H3)_rel'] = actual_data_final['A(H3)_abs']/actual_data_final['tested']
actual_data_final['B_rel'] = actual_data_final['B_abs']/actual_data_final['tested']
actual_data_final['A(H1)pdm09_total'] = (actual_data_final['A(H1)pdm09_rel']*actual_data_final['ari']).astype(int)
actual_data_final['A(H3)_total'] = (actual_data_final['A(H3)_rel']*actual_data_final['ari']).astype(int)
actual_data_final['B_total'] = (actual_data_final['B_rel']*actual_data_final['ari']).astype(int)



year = 2023
actual_data_final = actual_data_final[actual_data_final['year'] == year]
real_input = pd.read_csv('input/incidence_strains_spb_2_age_groups.csv')
real_input = real_input[real_input['Год'] != year]

add_df = pd.DataFrame(columns=['index', 'Год', 'Неделя', 'A(H1N1)pdm09_15 и ст.', 
                               'A(H3N2)_15 и ст.',
       'B_15 и ст.', 'Население 15 и ст.', 'A(H1N1)pdm09_15 и ст._rel',
       'A(H3N2)_15 и ст._rel', 'B_15 и ст._rel', 'A(H1N1)pdm09_0-14',
       'A(H1N1)pdm09_0-14_rel', 'Население 0-14', 'A(H3N2)_0-14',
       'A(H3N2)_0-14_rel', 'B_0-14', 'B_0-14_rel'])

add_df['Год'] = actual_data_final['year'] 
add_df['Население 15 и ст.'] = actual_data_final['population']
add_df['Неделя'] = actual_data_final['num_of_week']
add_df['A(H1N1)pdm09_15 и ст.'] = actual_data_final['A(H1)pdm09_total']
add_df['A(H3N2)_15 и ст.'] = actual_data_final['A(H3)_total']
add_df['B_15 и ст.'] = actual_data_final['B_total']
add_df['A(H1N1)pdm09_15 и ст._rel'] = actual_data_final['A(H1)pdm09_rel']
add_df['A(H3N2)_15 и ст._rel'] = actual_data_final['A(H3)_rel']
add_df['B_15 и ст._rel'] = actual_data_final['B_rel']
add_df = add_df.fillna(0)
add_df

real_input = pd.concat([real_input, add_df], ignore_index=True)
real_input['index'] = real_input.index
real_input = real_input.sort_values(by=['Год', 'Неделя'])
real_input.to_csv('input/incidence_strains_spb_2_age_groups.csv', index=False)
real_input
# actual_data_final[actual_data_final['year'] == year]

/tmp/ipykernel_154063/1760189163.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_data_preprocessed['date'] = actual_data_preprocessed['date'].str.replace(u'\xa0', u' ')


AttributeError: 'float' object has no attribute 'replace'